In [1]:
import pandas as pd

In [2]:
# Cargar el archivo CSV
data_trasversal_original = pd.read_csv(r"../data/final/2_trasversal/final_dataset_trasversal.csv")
data_trasversal_original = data_trasversal_original.dropna().reset_index(drop=True)
data_trasversal=data_trasversal_original
data_trasversal = data_trasversal.drop(columns=['conglome', 'vivienda','hogar','codperso','periodo','ocupinf'])
print(data_trasversal.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120987 entries, 0 to 120986
Data columns (total 99 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   dpto                     120987 non-null  int64  
 1   dominio                  120987 non-null  int64  
 2   tuvotrabajo              120987 non-null  int64  
 3   sector_trabajador        120987 non-null  int64  
 4   categoria_trabajador     120987 non-null  int64  
 5   trabajopara              120987 non-null  float64
 6   registropersonajuridica  120987 non-null  float64
 7   cuentassunat             120987 non-null  float64
 8   pagosueldo               120987 non-null  int64  
 9   pagosalario              120987 non-null  int64  
 10  pagocomision             120987 non-null  int64  
 11  pagodestajo              120987 non-null  int64  
 12  pagosubvencion           120987 non-null  int64  
 13  pagohonorarios           120987 non-null  int64  
 14  pago

In [3]:
########## Separamos del dataset las variables que serán útiles para la línea de vulnerabilidad y que son parte del modelo, pero
########## no son parte del modelo ni son predictores. Las guardamos en un dataset aparte.

data_trasversal= data_trasversal.drop(columns=['fac500a','ocu500','ingtra_n','resi'])

In [4]:
categoricas = ['sexo','estadocivil','tuvotrabajo','categoria_trabajador','sector_trabajador'
               ,'trabajopara','registropersonajuridica',
               'cuentassunat','tipocontrato','numpersonastrabajo','tuvootrotrabajo','normtrabaja',
               'disponiblehorastrabajar',
               'lenguamaterna','niveleduc','asiste_educ','usointernet','padece_enfer',
               'sintoma_malestar','enfermedad','recaida','accidente','notuvoenf',
               'puestosalud','centrosalud','centroopuesto','posta','hospital_minsa','hospital_seguro',
               'hospital_ffaa','consultorio','clinica','farmacia','domicilio','otro','no_busco',
               'no_dinero','lejos','demora_mucho','noconfia','nograve','remedio_caseros','no_seguro',
               'autoreceto','notiempo','maltrato','otro_motivo','essalud','seguroprivado',
               'eps','seguroffaa','sis','tipovivienda','materialparedes','materialpisos','materialtechos',
               'vivienda_status','viviendatitulo','agua_procedencia','conexionsshh','electricidad',
               'celular','internet','pobreza','dominio','dpto','ciiu_6c','area',
               'region','onp','afp',
               'pagosueldo','pagosalario','pagocomision','pagodestajo','pagosubvencion','pagohonorarios','pagoganancianegocio','pagoagropecuario','pagopropina','pagoespecie','pagootros','pagonorecibe']
# 'antepasadosconsidera','perteneceindig','registrosunat','deseaotrotrabajo','viviadistrito','agua_potable'
df_categoricas = data_trasversal[categoricas]
df_categoricas = df_categoricas.apply(lambda col: col.astype('int'))

df_continuas = data_trasversal.drop(columns=categoricas)

print("La cantidad de variables indep categoricas:")
print(df_categoricas.shape[1])
print("La cantidad de variables indep continuas:")
print(df_continuas.shape[1])



La cantidad de variables indep categoricas:
83
La cantidad de variables indep continuas:
12


In [5]:
# Crear dummies a partir de las variables categóricas
df_categoricas_encoded = pd.get_dummies(df_categoricas, columns=categoricas, drop_first=True, dummy_na=False)
print('La cantidad de variables indep categóricas después de haberse convertido en dummies es:')
print(df_categoricas_encoded.shape[1])

La cantidad de variables indep categóricas después de haberse convertido en dummies es:
228


In [6]:
import numpy as np
# Reemplazar ceros por un valor pequeño (por ejemplo, 1e-10)
df_continuas['ingtrabw'] = df_continuas['ingtrabw'].replace(0, 1e-10)

# Ahora aplicar el logaritmo natural (base e)
df_continuas['ingtrabw'] = np.log(df_continuas['ingtrabw'])

In [7]:
# El dataset final de variables independientes (categoricas convertidas a dummies + continuas)
df_indep_final = pd.concat([df_categoricas_encoded, df_continuas], axis=1)
print('La cantidad de variables indep totales (categoricas convertidas a dummies + continuas) es:')
print(df_indep_final.shape[1])

La cantidad de variables indep totales (categoricas convertidas a dummies + continuas) es:
240


In [8]:
import joblib

# Cargar el modelo
modelo = joblib.load('../output/modelos/XGBoost_con_SMOTE_model_best.joblib')

# Obtener las variables utilizadas
variables_usadas = modelo.feature_names_in_

df_xgb_linea = df_indep_final[variables_usadas]
df_xgb_linea = df_xgb_linea.copy()

df_xgb_linea.info()

# Predicción de probabilidades
y_proba = modelo.predict_proba(df_xgb_linea)[:, 1]

# Predicción con umbral 0.18
y_pred = (y_proba > 0.18).astype(int)

# Guardar resultados si quieres
df_xgb_linea['probabilidad'] = y_proba
df_xgb_linea['prediccion'] = y_pred

# Añadir las columnas al dataset grande usando el mismo índice
data_trasversal_original.loc[df_xgb_linea.index, "y_proba"] = y_proba
data_trasversal_original.loc[df_xgb_linea.index, "y_pred"]  = y_pred

ruta_salida = r"C:\projects\tesis_cyv\data\final\2_trasversal\4_linea\df_predicted_lineavulnerabilidad_xgb.csv"

data_trasversal_original.to_csv(ruta_salida, index=False, encoding='utf-8-sig')

print("Archivo guardado en:", ruta_salida)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120987 entries, 0 to 120986
Columns: 135 entries, sexo_2 to ratiodep
dtypes: bool(127), float64(4), int64(4)
memory usage: 22.0 MB
Archivo guardado en: C:\projects\tesis_cyv\data\final\2_trasversal\4_linea\df_predicted_lineavulnerabilidad_xgb.csv
